In [1]:
import pandas as pd
import numpy as np

import portfolio_tracker as pt

from datetime import datetime, timedelta
import pytz

# Configure plotly renderer
# pio.renderers.default = "notebook"

In [2]:
def create_test_portfolio():
    # Initialize portfolio
    market_db_name = 'market_data_example.db'
    portfolio_db_name = 'portfolio_example.db'
    initial_cash = 100000.0
    portfolio = pt.Portfolio(initial_deposit=initial_cash, market_data_path=market_db_name, storage_path=portfolio_db_name)

    # Test data for different symbols and exchanges
    test_data = [

        {
            'symbol': 'BTCUSDT',
            'exchange': 'BYBIT',
            'trades': [
                ('2024-01-28 10:00:00', 'buy', 41300, 0.52, 0.05),
                ('2024-10-15 10:00:00', 'buy', 67770.0, 0.04, 0.05),
                ('2024-11-22 10:00:00', 'sell', 98931.0, 0.2, 0.05),
            ]
        },
        {
            'symbol': 'ETHUSDT',
            'exchange': 'BYBIT',
            'trades': [
                ('2024-01-01 10:00:00', 'buy', 2350, 3, 0.05),
                ('2024-02-13 10:00:00', 'buy', 2471, 3, 0.05),
            ]
        },
        {
            'symbol': 'SOLUSDT',
            'exchange': 'BINANCE',
            'trades': [
                ('2024-01-08 08:00:00', 'buy', 97.5, 40, 0.05),
                ('2024-10-15 08:00:00', 'buy', 155.54, 30, 0.05),
            ]
        },
        {
            'symbol': 'SUIUSDT',
            'exchange': 'BINANCE',
            'trades': [
                ('2024-01-01 08:00:00', 'buy', 0.843, 15000, 0.05),
            ]
        },
        {
            'symbol': 'SDAIUSD',
            'exchange': 'PYTH',
            'trades': [
                ('2024-05-26 09:00:00', 'buy', 1.08, 25000, 0.05),   
            ]
        },
    ]

    # Add positions and execute trades
    for asset in test_data:
        symbol = asset['symbol']
        exchange = asset['exchange']

        # Add position
        portfolio.add_position(symbol, exchange)
        position = portfolio.get_position(symbol, exchange)

        # Execute trades
        for trade_date, trade_type, price, quantity, fee in asset['trades']:
            trade_datetime = datetime.strptime(trade_date, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.UTC)

            if trade_type == 'buy':
                position.add_to_position(quantity, price, trade_datetime, fee)
            else:
                position.reduce_position(quantity, price, trade_datetime, fee)

            # Save after each trade
            portfolio.storage.save_position(position)

    return portfolio

portfolio = create_test_portfolio()
summary = portfolio.get_position_summary()
summary

you are using nologin method, data you access may be limited
ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


,exchange,symbol,quantity,average_entry,current_price,market_value,realized_pnl,unrealized_pnl,total_pnl,pnl_percentage,max_drawdown,current_drawdown,7d_return,1m_return,4m_return,1y_return
0,BYBIT,BTCUSDT,0.36,43212.309643,101717.290000,36618.22440,11133.844971,21061.792929,32195.63790,135.389617,26.160286,4.150212,7.520289,4.558790,76.459315,120.593345
1,BYBIT,ETHUSDT,6.00,2411.705250,3680.900000,22085.40000,0.000000,7615.168500,7615.16850,52.626446,45.258409,9.446281,9.532992,-0.831144,54.087339,57.009529
2,BINANCE,SOLUSDT,70.00,122.435473,217.140000,15199.80000,0.000000,6629.316900,6629.31690,77.350563,38.231662,15.476839,11.766523,0.235424,60.097324,118.538647
3,BINANCE,SUIUSDT,15000.00,0.843422,5.070000,76050.00000,0.000000,63398.677500,63398.67750,501.122926,74.231473,4.162414,18.328004,32.103494,452.347750,535.099587
4,PYTH,SDAIUSD,25000.00,1.080540,1.133382,28334.53775,0.000000,1321.037750,1321.03775,4.890287,0.593578,0.000000,0.383416,0.830402,2.331287,4.455128


In [6]:
# Initialize your portfolio and visualizer
visualizer = pt.PortfolioVisualizer(portfolio)

main_dashboard = visualizer.create_dashboard(
    # benchmark_identifier='CRYPTOCAP:TOTAL',
    benchmark_identifier='BINANCE:BTCUSDT',
    show_individual_symbols=True,
    # start_date=datetime.now(pytz.UTC) - timedelta(days=30),
    # end_date=datetime.now(pytz.UTC) - timedelta(days=0),
)
main_dashboard.show()

In [4]:
portfolio.get_portfolio_summary()

{'current_value': 178287.96215,
 'total_investment': 78261.96857142857,
 'total_pnl': 100025.99357857143,
 'pnl_percentage': 127.80919698854643,
 'max_drawdown': 30.51780050553611,
 'current_drawdown': 1.0315145901599805,
 'sharpe_ratio': 2.213512621224874,
 'sortino_ratio': 4.583396372540516,
 '7d_return': 11.20985707499975,
 '1m_return': 12.75060629613792,
 '4m_return': 107.22406835211937,
 '1y_return': 259.48156200030354,
 'daily_volatility': 3.470772760097155,
 'annualized_volatility': 55.0968094826059}

In [5]:
# Create efficient frontier plot separately
ef_plot = visualizer.plot_efficient_frontier()
ef_plot.show()